# Finetuning LLAMA3:8B Com contéudos de medicina

Vamos utlizar o torchtune o torchtune é uma Pytorch-native lib para facilitar o finetuning de modelos de LLM.

In [ ]:
# !pip install wikipedia-api
# !pip install torchtune
# !pip install setuptools

In [ ]:
import wikipediaapi

def extract_wikipedia_data(pages, file_path):
    wiki_wiki = wikipediaapi.Wikipedia("Wikipedia/1.0", 'pt')

    with open(file_path, 'w', encoding='utf-8') as f:
        for page in pages:
            p_wiki = wiki_wiki.page(page)
            if p_wiki.exists():
                f.write(f"Title: {p_wiki.title}\n\n")
                f.write(f"{p_wiki.text}\n\n")
                f.write("="*80 + "\n\n")
            else:
                print(f"Page '{page}' does not exist on Wikipedia.")

# Lista de páginas a serem extraídas
pages = ["Medicina", "HIV/AIDS"]
# Caminho para salvar o arquivo de texto
file_path = "wikipedia_medicina_aids.txt"

extract_wikipedia_data(pages, file_path)

### 1. Vamos criar nosso proprio arquivo de configuração para o Torchtune.

In [1]:
!tune cp llama3/8B_full_single_device ./custom_config_wikipedia.yaml


### 2. Tudo que precisamos fazer agora é apontar para o nosso dataset.

```yaml
dataset:
  _component_: torchtune.datasets.text_completion_dataset
  source: text 
  column: text
  data_files: wikipedia_medicina_aids.txt
  split: train
seed: null
shuffle: True
batch_size: 2```

### 3. Agora podemos iniciar o fine tuning com o `torch run``

In [ ]:
!tune run lora_finetune_single_device --config custom_config_wikipedia.yaml